In [2]:
#imports
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import glob
import datetime
import cv2
from PIL import Image
random.seed(108)

In [6]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

c:\Curspaper\yolov7\yolov7


Cloning into 'yolov7'...


  Using cached protobuf-4.21.2-cp310-abi3-win_amd64.whl (524 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.0
    Uninstalling protobuf-4.23.0:
      Successfully uninstalled protobuf-4.23.0


In [3]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="mRs9vg8hQcE37xjAudq2")
project = rf.workspace("saratov-state-uni").project("number-detection-s85cr")
dataset = project.version(3).download("yolov7")


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\denzi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     -------------------------------------- 56.3/56.3 kB 368.4 kB/s eta 0:00:00
     ------------------------------------ 155.3/155.3 kB 165.7 kB/s eta 0:00:00
     ------------------------------------ 178.7/178.7 kB 399.3 kB/s eta 0:00:00
     -------------------------------------- 58.8/58.8 kB 239.3 kB/s eta 0:00:00
     -------------------------------------- 67.8/67.8 kB 245.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 54.5/54.5 kB 1.4 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=ae33f6d9b5435dbc303e9020e87d6f48d8e204118e59aafca6d99a81a782e56c
  Stored in directory: c:\users\denzi\appdata\local\pip\cache\wheels\f0\e4\9c\b3ed593784fe0147db216173b4046fce93fae7130b29e4464e
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
     

Extracting Dataset Version Zip to number-detection-3 in yolov7pytorch:: 100%|██████████| 1388/1388 [00:01<00:00, 738.07it/s]


In [5]:
!pip install tensorrt

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [56 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib
      creating build\lib\tensorrt
      copying tensorrt\__init__.py -> build\lib\tensorrt
      running egg_info
      writing tensorrt.egg-info\PKG-INFO
      writing dependency_links to tensorrt.egg-info\dependency_links.txt
      writing requirements to tensorrt.egg-info\requires.txt
      writing top-level names to tensorrt.egg-info\top_level.txt
      reading manifest file 'tensorrt.egg-info\SOURCES.txt'
      adding license file 'LICENSE.txt'
      writing manifest file 'tensorrt.egg-info\SOURCES.txt'
      C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build a


  Using cached tensorrt-8.6.1.tar.gz (16 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorrt
Failed to build tensorrt


In [5]:
!wandb login e478cc6c0a4343a9912bfe3a45254bbf4c6139c1

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\denzi/.netrc


In [7]:
%%writefile cfg/training/yolov7_pothole-tiny.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
 
# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32
 
# yolov7-tiny backbone
backbone:
  # [from, number, module, args] c2, k=1, s=1, p=None, g=1, act=True
  [[-1, 1, Conv, [32, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 0-P1/2 
  
  [-1, 1, Conv, [64, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 1-P2/4   
  
  [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 7
  
  [-1, 1, MP, []],  # 8-P3/8
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 14
  
  [-1, 1, MP, []],  # 15-P4/16
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 21
  
  [-1, 1, MP, []],  # 22-P5/32
  [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [512, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 28
  ]
 
# yolov7-tiny head
head:
  [[-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, SP, [5]],
  [-2, 1, SP, [9]],
  [-3, 1, SP, [13]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -7], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 37
  
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [21, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P4
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 47
  
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [14, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P3
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 57
  
  [-1, 1, Conv, [128, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, 47], 1, Concat, [1]],
  
  [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 65
  
  [-1, 1, Conv, [256, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, 37], 1, Concat, [1]],
  
  [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [[-1, -2, -3, -4], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 73
      
  [57, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [65, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
  [73, 1, Conv, [512, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
 
  [[74,75,76], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

Writing cfg/training/yolov7_pothole-tiny.yaml


In [12]:
!python train.py --epochs 50 --batch-size 8 \
--data number-detection-3/data.yaml --img 640 640 --cfg cfg/training/yolov7_pothole-tiny.yaml \
--weights 'yolov7-tiny.pt' --name yolov7_tiny_fixed_res --hyp data/hyp.scratch.tiny.yaml

YOLOR  v0.1-122-g3b41c2c torch 2.0.1+cpu CPU

Namespace(weights="'yolov7-tiny.pt'", cfg='cfg/training/yolov7_pothole-tiny.yaml', data='number-detection-3/data.yaml', hyp='data/hyp.scratch.tiny.yaml', epochs=50, batch_size=8, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7_tiny_fixed_res', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\yolov7_tiny_fixed_res3', total_batch_size=8)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epo


autoanchor: Analyzing anchors... anchors/target = 5.06, Best Possible Recall (BPR) = 1.0000
                 all         139           0           0           0           0           0
                 all         139         194      0.0176     0.00515    9.87e-05    3.95e-05
                 all         139         194    9.66e-05     0.00515    6.59e-07    1.32e-07
                 all         139         194    9.62e-05     0.00515    6.76e-07    6.76e-08
                 all         139           0           0           0           0           0
                 all         139         194    0.000207     0.00515    2.48e-06    2.48e-07
                 all         139         194    0.000993     0.00515     1.1e-05    2.21e-06
                 all         139         194    7.19e-05      0.0155    2.53e-06    8.25e-07
                 all         139           0           0           0           0           0
                 all         139         194    0.000104      0.0206  

     33/49        0G   0.02893  0.006266         0    0.0352         9       640:  22%|██▏       | 13/60 [01:36<05:49,  7.44s/it]
     33/49        0G   0.02807  0.006282         0   0.03435        14       640:  22%|██▏       | 13/60 [01:44<05:49,  7.44s/it]
     33/49        0G   0.02807  0.006282         0   0.03435        14       640:  23%|██▎       | 14/60 [01:44<05:41,  7.42s/it]
     33/49        0G    0.0286  0.006296         0   0.03489        13       640:  23%|██▎       | 14/60 [01:51<05:41,  7.42s/it]
     33/49        0G    0.0286  0.006296         0   0.03489        13       640:  25%|██▌       | 15/60 [01:51<05:36,  7.47s/it]
     33/49        0G   0.03079  0.006179         0   0.03697         9       640:  25%|██▌       | 15/60 [01:59<05:36,  7.47s/it]
     33/49        0G   0.03079  0.006179         0   0.03697         9       640:  27%|██▋       | 16/60 [01:59<05:27,  7.44s/it]
     33/49        0G   0.03015  0.006121         0   0.03627        11       640:  27%|██▋

In [7]:
%cd yolov7/
!python test.py --task test --data number-detection-3/data.yaml --weights runs/train/yolov7_tiny_fixed_res3/weights/best.pt --conf 0.25 --name yolo_road_det

[WinError 2] Не удается найти указанный файл: 'yolov7/'
c:\Curspaper\yolov7
Namespace(weights=['runs/train/yolov7_tiny_fixed_res3/weights/best.pt'], data='number-detection-3/data.yaml', batch_size=32, img_size=640, conf_thres=0.25, iou_thres=0.65, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolo_road_det', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all          69          92       0.619       0.282       0.224      0.0952
Speed: 396.2/1.2/397.5 ms inference/NMS/total per 640x640 image at batch-size 32
Results saved to runs\test\yolo_road_det


YOLOR  v0.1-122-g3b41c2c torch 2.0.1+cpu CPU

C:\Users\denzi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS

Scanning images:   0%|          | 0/69 [00:00<?, ?it/s]
test: Scanning 'number-detection-3\test\labels' images and labels... 5 found, 0 missing, 0 empty, 0 corrupted:   7%|▋         | 5/69 [00:00<00:01, 46.43it/s]
test: Scanning 'number-detection-3\test\labels' images and labels... 15 found, 0 missing, 0 empty, 0 corrupted:  22%|██▏       | 15/69 [00:00<00:00, 72.82it/s]
test: Scanning 'number-detection-3\test\labels' images and labels... 24 found, 0 missing, 0 empt

In [8]:
detections_dir = "runs/test/yolo_road_det/"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))

NameError: name 'os' is not defined

In [9]:
%%writefile cfg/training/yolov7_bus.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
 
# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32
 
# yolov7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
  
  [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2     
  [-1, 1, Conv, [64, 3, 1]],
  
  [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4 
  [-1, 1, Conv, [64, 1, 1]],
  [-2, 1, Conv, [64, 1, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]],  # 11
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [128, 1, 1]],
  [-3, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 16-P3/8 
  [-1, 1, Conv, [128, 1, 1]],
  [-2, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [512, 1, 1]],  # 24
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [256, 1, 1]],
  [-3, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 29-P4/16 
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [1024, 1, 1]],  # 37
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [512, 1, 1]],
  [-3, 1, Conv, [512, 1, 1]],
  [-1, 1, Conv, [512, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 42-P5/32 
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [1024, 1, 1]],  # 50
  ]
 
# yolov7 head
head:
  [[-1, 1, SPPCSPC, [512]], # 51
  
  [-1, 1, Conv, [256, 1, 1]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [37, 1, Conv, [256, 1, 1]], # route backbone P4
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]], # 63
  
  [-1, 1, Conv, [128, 1, 1]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [24, 1, Conv, [128, 1, 1]], # route backbone P3
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [128, 1, 1]],
  [-2, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [128, 1, 1]], # 75
      
  [-1, 1, MP, []],
  [-1, 1, Conv, [128, 1, 1]],
  [-3, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 2]],
  [[-1, -3, 63], 1, Concat, [1]],
  
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]], # 88
      
  [-1, 1, MP, []],
  [-1, 1, Conv, [256, 1, 1]],
  [-3, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 2]],
  [[-1, -3, 51], 1, Concat, [1]],
  
  [-1, 1, Conv, [512, 1, 1]],
  [-2, 1, Conv, [512, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [512, 1, 1]], # 101
  
  [75, 1, RepConv, [256, 3, 1]],
  [88, 1, RepConv, [512, 3, 1]],
  [101, 1, RepConv, [1024, 3, 1]],
 
  [[102,103,104], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

Writing cfg/training/yolov7_bus.yaml


In [1]:
%cd yolov7/
!python train.py --epochs 100 --workers 4  --batch-size 8 --data number-detection-3/data.yaml \
--img 640 640 --cfg cfg/training/yolov7_bus.yaml --weights 'yolov7_training.pt' \
--name yolov7_pothole_fixed_res --hyp data/hyp.scratch.custom.yaml

c:\Curspaper\yolov7


In [2]:
!python detect.py --save-txt --source number-detection-3/test/images --weights best.pt --conf 0.25 --name yolo_road_det

Traceback (most recent call last):
  File "c:\Curspaper\yolov7\detect.py", line 10, in <module>
    from models.experimental import attempt_load
  File "c:\Curspaper\yolov7\models\experimental.py", line 6, in <module>
    from yolov7.models.common import Conv, DWConv
ModuleNotFoundError: No module named 'yolov7'


In [4]:
detect_path = 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2'; 
labels = glob.glob(os.path.join('C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels', '*.txt')) 
annotation_file = random.choice(labels) 
print(annotation_file)

C:\Curspaper\yolov7\runs\detect\yolo_road_det2\labels\image-164-26_jpg.rf.7c582404ebc1567c88b52a44fa20c56c.txt


In [3]:
#get targer images
def plot_bounding_box(image_file, image, annotation_list, i): 
    annotations = np.array(annotation_list) 
    w, h = image.size 
    directory = image_file[0:len(image_file)-3]
    if not os.path.exists(directory):
        os.makedirs(directory)
    transformed_annotations = np.copy(annotations) 
    transformed_annotations[:,[0,2]] = annotations[:,[0,2]] * w 
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * h  
    
    transformed_annotations[:,0] = transformed_annotations[:,0] - (transformed_annotations[:,2] / 2) - 3
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2) - 3
    transformed_annotations[:,2] = transformed_annotations[:,0] + transformed_annotations[:,2] + 3
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3] + 3
    
    for ann in transformed_annotations: 
        x0, y0, x1, y1 = ann 
        cropped_image = image.crop((x0,y0, x1, y1))
        cropped_image.save(os.path.join(directory,  str(i) + '.jpg'))
        
    
# Get any random annotation file  
detect_path = 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2'; 
labels = glob.glob(os.path.join('C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels', '*.txt')) 
annotation_file = 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-792-47_jpg.rf.78cb01307da1dc24895602f42c6c96db.txt'

with open(annotation_file, 'r') as f:
  result = list()
  for line in f: 
    annotation_list = line.split("\n")[:-1] 
    annotation_list = [x.split(" ") for x in annotation_list] 
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
    result.append(annotation_list)
image_file = annotation_file.replace("labels\\", "")
image_file = image_file[0:len(image_file)-3] 
image_file += "jpg" 
assert os.path.exists(image_file) 
 
#Load the image 
image = Image.open(image_file) 
 
#Plot the Bounding Box 
i = 1
for bbox in result:
    plot_bounding_box(image_file, image, [bbox[::][0][1::]], i)
    i += 1

['C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-1-53_jpg.rf.acf7c6c8961a2e16284a42187fa13aff.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-109-30_jpg.rf.3f6dbc2f715ddc4492436ba5dd2be581.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-117-90_jpg.rf.9aa5c8529f6f117cd49549ae5245c539.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-119-90_jpg.rf.482d5e2e33e1a288dce8f419f25d0acb.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-135-90_jpg.rf.d8c2042ed50d1f8a55771a0b6672e92f.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-141-90_jpg.rf.914c54a605361ccc405755d16122ad62.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-143-90_jpg.rf.e04010060e0526a528031afd36976350.txt', 'C:\\Curspaper\\yolov7\\runs\\detect\\yolo_road_det2\\labels\\image-156-246_jpg.rf.e6d2048ec2de1338b365d525d334f4aa.txt', 'C:\\Curspaper\\yolov7\\runs\\de

ValueError: invalid literal for int() with base 10: '0.340625'